<a target="_blank" href="https://colab.research.google.com/github/chasubeen/Kwater/tree/subeen">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd

import os
import chardet

In [ ]:
pd.set_option('display.float_format', None) # 과학적 표기법 x

# **1. EDA**

# **2. 데이터 스케일링**

- train/test split 후 진행해야 함
- EDA -> train/test split -> 스케일링 진행

## **2-1. 표준화(StandardScaler)**

- 데이터가 X_train, y_train, X_test, y_test로 분할되었다고 가정

In [ ]:
from sklearn.preprocessing import StandardScaler

std = StandardScaler()
std_X_train = std.fit_transform(X_train)
std_X_test = std.transform(X_test)

In [ ]:
# 학습용 데이터

pd.DataFrame(std_X_train, columns = X_train.columns)

In [ ]:
# 평가용 데이터

pd.DataFrame(std_X_test, columns = X_test.columns)

### **2-2. 정규화(MinMaxScaler)**

In [ ]:
from sklearn.preprocessing import MinMaxScaler

mms = StandardScaler()
mms_X_train = mms.fit_transform(X_train)
mms_X_test = mms.transform(X_test)

In [ ]:
# 학습용 데이터

pd.DataFrame(mms_X_train, columns = X_train.columns)

In [ ]:
# 평가용 데이터

pd.DataFrame(mms_X_test, columns = X_test.columns)

### **2-3. 표준정규화(RobustScaler)**
- 데이터의 중앙값(median)과 IQR(Interquartile Range)을 사용하여 스케일링을 수행
  - 이상치에 영향을 덜 받음

In [ ]:
from sklearn.preprocessing import RobustScaler

robust = RobustScaler()
robust_X_train = robust.fit_transform(X_train)
robust_X_test = robust.transform(X_test)

In [ ]:
# 학습용 데이터
pd.DataFrame(robust_X_train, columns = X_train.columns)

In [ ]:
# 평가용 데이터
pd.DataFrame(robust_X_test, columns = X_test.columns)